# LMNA 

The data in this workbook were derived from Table 1 of [Bonne G, et al (2000) Clinical and molecular genetic spectrum of autosomal dominant Emery-Dreifuss muscular dystrophy due to mutations of the lamin A/C gene. Ann Neurol 48(2):170-80. PMID:10939567](https://pubmed.ncbi.nlm.nih.gov/10939567/).

Note that we will code the following two features as excluded in all individuals based on knowledge external to the above cited publication.

- Excluded: Progeroid facial appearance HP:0005328 (see PMID:29633897 and OMIM)
- Excluded: Lipodystrophy HP:0009125 (see PMID:29633897 and OMIM)


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.65


In [2]:
PMID = "PMID:10939567"
title = "Clinical and molecular genetic spectrum of autosomal dominant Emery-Dreifuss muscular dystrophy due to mutations of the lamin A/C gene"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_csv("input/Bonne2000.tsv", sep='\t')
df.head()

,Patient,Age,Onset,First Symptoms,Course,Walks(age [yr]),Severity,CK,Wasting,Weakness in Upper Limbs,Weakness in Lower Limbs,Neck,Contracture,Spine,Electrocardiography,Echocardiography,NM_170707.4
0,EMD1 III-2,51,Childhood,"Waddling gait,difficulty in running",Slowly progressive,YD,++,na,Scapuloperoneal,Proximal,NaN,stiff,"E,A",Rigid,Pacemaker,Ventricular dysfunction,c.16C>T
1,EMD1 III-20,61,Childhood,"Waddling gait,difficulty in running",Slowly progressive,WCB(55),++,2,Scapuloperoneal,Proximal,NaN,stiff,"E,A",Rigid,Pacemaker,Ventricular dysfunction,c.16C>T
2,EMD1 IV-13,48,P15Y,"AT contractures, difficulty in running",Slowly progressive,YD,+,2,Scapuloperoneal,Distal,Proximal,N,"E,A",Rigid,Arrhythmia,N,c.16C>T
3,EMD1 IV-14,46,na,Difficulty in climbing,Slowly progressive,YD,+,na,Scapuloperoneal,Proximal,NaN,stiff,A,Rigid,Arrhythmia,N,c.16C>T
4,EMD1 IV-24,41,P15Y,AT contractures,Slowly progressive,YD,+,2,Scapuloperoneal,Proximal,NaN,stiff,"E,A",Rigid,Pacemaker,N,c.16C>T


In [4]:
# Make the approximate assumption that childhood onset is P2Y
def iso_age(onset):
    if onset.startswith("P"):
        return onset
    elif onset == "Childhood":
        return "P2Y"
    elif onset == "na":
        return "n/a"
    else:
        return f"Could not map{onset}"

df['iso_onset'] = df["Onset"].apply(lambda x: iso_age(x))
ageMapper = AgeColumnMapper.iso8601(column_name='iso_onset')
#ageMapper.preview_column(df['iso_onset'])
sexMapper = SexColumnMapper.not_provided()

In [5]:
def indiv_id(patient):
    individual_id = patient
    return individual_id.replace(" ", "_")
    
df['individual_id'] = df["Patient"].apply(lambda x: indiv_id(x))

In [6]:
#res = OptionColumnMapper.autoformat(df=df, concept_recognizer=hpo_cr)
#print(res)
column_mapper_list = list()

In [7]:
first_symptoms_d = {'Waddling gait': 'Waddling gait',
 'difficulty in running': 'Difficulty running',
 'AT contractures': 'Achilles tendon contracture',
 'Difficulty in climbing': 'Difficulty climbing stairs',
 'Difficulty in walking': 'Difficulty walking',
 'Difficulty in running': 'Difficulty running',
 'Elbow flexion': 'Elbow flexion contracture',
 'Weakness': 'Muscle weakness',
 'positive Gower sign': 'Gowers sign',
 'Frequent falls': 'Frequent falls',
 'Positive Gower sign': 'Gowers sign',
 #'Difficulty in getting up': 'PLACEHOLDER',
 'Falling': 'Frequent falls',
 'fatigue': 'Fatigue',
 'Fatigue': 'Fatigue',
 'foot deformity': 'Abnormal foot morphology',
 'Minimal wasting': 'Skeletal muscle atrophy',
 'Delayed motor development': 'Motor delay'}
firstsymptomsMapper = OptionColumnMapper(column_name="First  Symptoms", concept_recognizer=hpo_cr, option_d=first_symptoms_d)
column_mapper_list.append(firstsymptomsMapper)
firstsymptomsMapper.preview_column(df)

,mapping,count
0,Waddling gait (HP:0002515) (observed),5
1,Difficulty running (HP:0009046) (observed),9
2,Achilles tendon contracture (HP:0001771) (observed),4
3,Difficulty climbing stairs (HP:0003551) (observed),3
4,Difficulty walking (HP:0002355) (observed),6
5,Elbow flexion contracture (HP:0002987) (observed),2
6,Muscle weakness (HP:0001324) (observed),1
7,Gowers sign (HP:0003391) (observed),3
8,Frequent falls (HP:0002359) (observed),6
9,Fatigue (HP:0012378) (observed),7


In [8]:
walks_d = {'YD': 'Difficulty walking',
 'WCB(55)': 'Loss of ambulation',
 'WCB(13)': 'Loss of ambulation',
 'WCB(8)': 'Loss of ambulation'}
walksMapper = OptionColumnMapper(column_name='Walks(age  [yr])',concept_recognizer=hpo_cr, option_d=walks_d)
column_mapper_list.append(walksMapper)
walksMapper.preview_column(df)

,mapping,count
0,Difficulty walking (HP:0002355) (observed),11
1,Loss of ambulation (HP:0002505) (observed),4


In [9]:
ck_d = {
        '2': 'Elevated circulating creatine kinase concentration',
        '3': 'Elevated circulating creatine kinase concentration',
        '4': 'Highly elevated creatine kinase',
        '5': 'Highly elevated creatine kinase',
        '6': 'Highly elevated creatine kinase',
        '7': 'Highly elevated creatine kinase',
        '8': 'Highly elevated creatine kinase',
         '15': 'Highly elevated creatine kinase',
         '14': 'Highly elevated creatine kinase'}
excluded ={'N': 'Elevated circulating creatine kinase concentration',}
ckMapper = OptionColumnMapper(column_name="CK",concept_recognizer=hpo_cr, option_d=ck_d, excluded_d=excluded)
column_mapper_list.append(ckMapper)
ckMapper.preview_column(df)

,mapping,count
0,Elevated circulating creatine kinase concentration (HP:0003236) (observed),13
1,Highly elevated creatine kinase (HP:0030234) (observed),16
2,Elevated circulating creatine kinase concentration (HP:0003236) (excluded),2


In [10]:
wasting_d = {'Scapuloperoneal': 'Scapuloperoneal amyotrophy',
 'Scapular': 'Amyotrophy involving the shoulder musculature',
 'Mild distal': 'Distal amyotrophy',
 'pes cavus': 'Pes cavus',
 'Diffuse wasting': 'Distal amyotrophy',
 'preddistal': 'Distal amyotrophy',
 'Pseudohypertrophy calves': 'Calf muscle pseudohypertrophy',
 'Both arms': 'Upper limb amyotrophy',
 'Humeroperoneal': 'Proximal amyotrophy',
 'bilateral pes equinus': 'Talipes equinovarus',
 'Mild proximal': 'Proximal amyotrophy',
 'Proximal': 'Proximal amyotrophy',
 'scapular winging': 'Scapular winging',
 'Diffuse': 'Generalized amyotrophy',
 #'Pseudohypertrophy in deltoideus': 'PLACEHOLDER', no term
 'wasting shoulder': 'Amyotrophy involving the shoulder musculature',
 'wasting leg': 'Lower limb amyotrophy',
 'Upper arms': 'Proximal upper limb amyotrophy',
 'glutei': 'Proximal lower limb amyotrophy',
 'Proximal muscle': 'Proximal amyotrophy',
 'upper arm': 'Upper limb amyotrophy',
 'leg': 'Lower limb amyotrophy',
 'Distal': 'Distal amyotrophy',
 'pes equinovarus': 'Talipes equinovarus',
 'legs': 'Lower limb amyotrophy',
 'Humeral': 'Proximal upper limb amyotrophy',
 'posterior leg': 'Lower limb amyotrophy',
 'upper leg': 'Proximal lower limb amyotrophy',
 'bilateral pes equinus': 'Bilateral talipes equinovarus'}
wastingMapper = OptionColumnMapper(column_name='Wasting',concept_recognizer=hpo_cr, option_d=wasting_d)
column_mapper_list.append(wastingMapper)
wastingMapper.preview_column(df)

,mapping,count
0,Scapuloperoneal amyotrophy (HP:0003697) (observed),13
1,Amyotrophy involving the shoulder musculature (HP:0001465) (observed),10
2,Distal amyotrophy (HP:0003693) (observed),6
3,Pes cavus (HP:0001761) (observed),2
4,Calf muscle pseudohypertrophy (HP:0003707) (observed),1
5,Upper limb amyotrophy (HP:0009129) (observed),3
6,Proximal amyotrophy (HP:0007126) (observed),4
7,Bilateral talipes equinovarus (HP:0001776) (observed),2
8,Scapular winging (HP:0003691) (observed),1
9,Generalized amyotrophy (HP:0003700) (observed),2


In [11]:
weakUL_d = {'Proximal': 'Proximal muscle weakness in upper limbs',
 'Distal': 'Upper limb muscle weakness',
 'Mildly proximal': 'Proximal muscle weakness in upper limbs'}
weakULMapper = OptionColumnMapper(column_name='Weakness  in Upper Limbs',concept_recognizer=hpo_cr, option_d=weakUL_d)
column_mapper_list.append(weakULMapper)
weakULMapper.preview_column(df)

,mapping,count
0,Proximal muscle weakness in upper limbs (HP:0008997) (observed),35
1,Upper limb muscle weakness (HP:0003484) (observed),18


In [12]:
weakLL_d = {
 'Proximal': 'Proximal muscle weakness in lower limbs',
 'Distal': 'Distal lower limb muscle weakness',
 }
weakLLMapper = OptionColumnMapper(column_name='Weakness  in Lower  Limbs',concept_recognizer=hpo_cr, option_d=weakLL_d)
column_mapper_list.append(weakLLMapper)
weakLLMapper.preview_column(df)

,mapping,count
0,Proximal muscle weakness in lower limbs (HP:0008994) (observed),21
1,Distal lower limb muscle weakness (HP:0009053) (observed),21


In [13]:
neck_d = {'stiff': 'Stiff neck',}
neckMapper = OptionColumnMapper(column_name='Neck',concept_recognizer=hpo_cr, option_d=neck_d)
column_mapper_list.append(neckMapper)
neckMapper.preview_column(df)

,mapping,count
0,Stiff neck (HP:0025258) (observed),36


In [14]:
# A: ankles;  E: elbow;  F: finger;  H: hips;  K: knee;  W: wrist;  N: normal;  
contracture_d = {'E': 'Elbow contracture',
 'A': 'Ankle contracture',
 'K': 'Knee contracture',
 'H': 'Hip contracture',
 'W': 'Wrist flexion contracture',
 'F': 'Finger joint contracture'}
excluded = {'N':'Joint contracture' }
contractureMapper = OptionColumnMapper(column_name='Contracture',concept_recognizer=hpo_cr, option_d=contracture_d, excluded_d=excluded)
column_mapper_list.append(contractureMapper)
contractureMapper.preview_column(df)

,mapping,count
0,Elbow contracture (HP:0034391) (observed),39
1,Ankle contracture (HP:0034677) (observed),39
2,Knee contracture (HP:0034671) (observed),5
3,Hip contracture (HP:0003273) (observed),13
4,Wrist flexion contracture (HP:0001239) (observed),3
5,Finger joint contracture (HP:0034681) (observed),2


In [15]:
spine_d = {'Rigid': 'Spinal rigidity',
 'Scoliosis': 'Scoliosis',
 'thoracic kyphosis': 'Thoracic kyphosis',
 'hyperext lordosis': 'Hyperlordosis'}
spineMapper = OptionColumnMapper(column_name='Spine',concept_recognizer=hpo_cr, option_d=spine_d)
column_mapper_list.append(spineMapper)
spineMapper.preview_column(df)

,mapping,count
0,Spinal rigidity (HP:0003306) (observed),36
1,Scoliosis (HP:0002650) (observed),5
2,Thoracic kyphosis (HP:0002942) (observed),1
3,Hyperlordosis (HP:0003307) (observed),1


In [16]:
# Six  patients  had  a  pacemaker  inserted  (age  range,  35– 61years),  
# and  1  patient  (Spo1)  had  an  implantable  defibrillator  inserted  for  
# severe  ventricular  dysrhythmias during electrophysiological testing.
ecg_d = {'Pacemaker': 'Ventricular arrhythmia',
 'Arrhythmia': 'Arrhythmia'}
ecgMapper = OptionColumnMapper(column_name='Electrocardiography',concept_recognizer=hpo_cr, option_d=ecg_d)
column_mapper_list.append(ecgMapper)
ecgMapper.preview_column(df)

,mapping,count
0,Ventricular arrhythmia (HP:0004308) (observed),6
1,Arrhythmia (HP:0011675) (observed),23


In [17]:
echo_d = {'Ventricular dysfunction': 'Abnormal left ventricular function'}
excluded = {'N': 'Abnormal left ventricular function'}
echoMapper = OptionColumnMapper(column_name='Echocardiography',concept_recognizer=hpo_cr, option_d=echo_d, excluded_d=excluded)
column_mapper_list.append(echoMapper)
echoMapper.preview_column(df)

,mapping,count
0,Abnormal left ventricular function (HP:0005162) (observed),9
1,Abnormal left ventricular function (HP:0005162) (excluded),27


# Variants

We mapped the variants to HGNC nomenclature as follows

- EMD1 CAG -> TAG  Q6Stop:  NM_170707.4(LMNA):c.16C>T (p.Gln6Ter) 
- Spo3 TAC -> TGC  Y45C: NM_170707.4(LMNA):c.134A>G (p.Tyr45Cys) 
- Spo27 CGC -> CCC  R50P: NM_170707.4(LMNA):c.149G>C (p.Arg50Pro) 
- Spo104 ATC ->AGC I63S: NM_170707.4(LMNA):c.188T>G (p.Ile63Ser) 
- Spo20 delGAG del112: NM_170707.4(LMNA):c.334_336del (p.Glu112del)
- EMD3 CAT -> CCT H222P: NM_170707.4(LMNA):c.665A>C (p.His222Pro)
- Spo70 GGG ->GAG G232E: NM_170707.4(LMNA):c.695G>A (p.Gly232Glu) 
- Spo89 CGG -> CAG R249Q: NM_170707.4(LMNA):c.746G>A (p.Arg249Gln)
- Spo1 delAAG delK261: NM_170707.4:c.781_783del/NP_733821.1:p.(Lys261del)
- Spo2 CAG ->CCG Q294P: NM_170707.4(LMNA):c.881A>C (p.Gln294Pro)
- Spo11 GAG -> AAG E358K: NM_170707.4(LMNA):c.1072G>A (p.Glu358Lys)
- Spo69 GAG -> AAG E358K: NM_170707.4(LMNA):c.1072G>A (p.Glu358Lys)
- Spo10 ATG -> AAG M371K: NM_170707.4(LMNA):c.1112T>A (p.Met371Lys)
- EMD5 AG/G -> AA/G R386K+intron 6: NM_170707.4(LMNA):c.1157G>A (p.Arg386Lys)
- Spo47 CGG -> TGG R453W: NM_170707.4(LMNA):c.1357C>T (p.Arg453Trp)
- Spo37 CGG -> TGG R453W: NM_170707.4(LMNA):c.1357C>T (p.Arg453Trp)
- EMD2 CGG -> TGG R453W: NM_170707.4(LMNA):c.1357C>T (p.Arg453Trp)
- Spo78 AAC -> AAA N456K: NM_170707.4(LMNA):c.1368C>A (p.Asn456Lys)
- Spo21 TGG -> TCG W520S: NM_170707.4(LMNA):c.1559G>C (p.Trp520Ser)
- EMD6 CGT -> CCT R527P: NM_170707.4(LMNA):c.1580G>C (p.Arg527Pro)
- EMD4 CGT -> CCT R527P: NM_170707.4(LMNA):c.1580G>C (p.Arg527Pro)
- Spo28 ACG -> AAG T528K: NM_170707.4(LMNA):c.1583C>A (p.Thr528Lys)
- Spo29 ACG -> AAG T528K: NM_170707.4(LMNA):c.1583C>A (p.Thr528Lys)

In [18]:
LMNA_transcript = "NM_005572.4"
vman = VariantManager(df=df, individual_column_name="individual_id",
                      allele_1_column_name='NM_170707.4', gene_symbol="LMNA",transcript=LMNA_transcript)

In [20]:
var_d = vman.get_variant_d()
varMapper = VariantColumnMapper(variant_d=var_d, variant_column_name='NM_170707.4', default_genotype='heterozygous')
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="individual_id", 
                        age_of_onset_mapper=ageMapper, 
                        variant_mapper=varMapper,
                        sexmapper=sexMapper,
                        metadata=metadata)
omim_id = "OMIM:181350"
omim_label = "Emery-Dreifuss muscular dystrophy 2, autosomal dominant" 
edmd2 = Disease(disease_id=omim_id, disease_label=omim_label)
encoder.set_disease(disease=edmd2)

In [21]:
individuals = encoder.get_individuals()
# We code the following terms as excluded (see top of notebook)
for i in individuals:
    progeroid = HpTerm(hpo_id="HP:0005328", label="Progeroid facial appearance", observed=False)
    lipodystrophy = HpTerm(hpo_id="HP:0009125", label="Lipodystrophy", observed=False)
    i.add_hpo_term(progeroid)
    i.add_hpo_term(lipodystrophy)

Could not parse the following as ISO8601 ages: n/a (n=1)


In [22]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,22


In [23]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
EMD1_III-2 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.16C>T (heterozygous),Waddling gait (HP:0002515); Difficulty running (HP:0009046); Difficulty walking (HP:0002355); Scapuloperoneal amyotrophy (HP:0003697); Proximal muscle weakness in upper limbs (HP:0008997); Stiff neck (HP:0025258); Elbow contracture (HP:0034391); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); Ventricular arrhythmia (HP:0004308); Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
EMD1_III-20 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.16C>T (heterozygous),Waddling gait (HP:0002515); Difficulty running (HP:0009046); Loss of ambulation (HP:0002505); Elevated circulating creatine kinase concentration (HP:0003236); Scapuloperoneal amyotrophy (HP:0003697); Proximal muscle weakness in upper limbs (HP:0008997); Stiff neck (HP:0025258); Elbow contracture (HP:0034391); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); Ventricular arrhythmia (HP:0004308); Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
EMD1_IV-13 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.16C>T (heterozygous),Achilles tendon contracture (HP:0001771); Difficulty running (HP:0009046); Difficulty walking (HP:0002355); Elevated circulating creatine kinase concentration (HP:0003236); Scapuloperoneal amyotrophy (HP:0003697); Upper limb muscle weakness (HP:0003484); Proximal muscle weakness in lower limbs (HP:0008994); Elbow contracture (HP:0034391); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); Arrhythmia (HP:0011675); excluded: Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
EMD1_IV-14 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.16C>T (heterozygous),Difficulty climbing stairs (HP:0003551); Difficulty walking (HP:0002355); Scapuloperoneal amyotrophy (HP:0003697); Proximal muscle weakness in upper limbs (HP:0008997); Stiff neck (HP:0025258); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); Arrhythmia (HP:0011675); excluded: Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
EMD1_IV-24 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.16C>T (heterozygous),Achilles tendon contracture (HP:0001771); Difficulty walking (HP:0002355); Elevated circulating creatine kinase concentration (HP:0003236); Scapuloperoneal amyotrophy (HP:0003697); Proximal muscle weakness in upper limbs (HP:0008997); Stiff neck (HP:0025258); Elbow contracture (HP:0034391); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); Ventricular arrhythmia (HP:0004308); excluded: Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP:0009125)
Spo3 (UNKNOWN; n/a),"Emery-Dreifuss muscular dystrophy 2, autosomal dominant (OMIM:181350)",NM_005572.4:c.134A>G (heterozygous),Difficulty walking (HP:0002355); Elevated circulating creatine kinase concentration (HP:0003236); Scapuloperoneal amyotrophy (HP:0003697); Proximal muscle weakness in upper limbs (HP:0008997); Proximal muscle weakness in lower limbs (HP:0008994); Distal lower limb muscle weakness (HP:0009053); Stiff neck (HP:0025258); Elbow contracture (HP:0034391); Knee contracture (HP:0034671); Ankle contracture (HP:0034677); Spinal rigidity (HP:0003306); excluded: Abnormal left ventricular function (HP:0005162); excluded: Progeroid facial appearance (HP:0005328); excluded: Lipodystrophy (HP

In [24]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                              metadata=metadata)

We output 41 GA4GH phenopackets to the directory phenopackets
